## AIHub Json Parsing

### Development Environment

In [ ]:
%pip install kss==3.7.3

KSS Argument Error: Restart Jupyter Kernel Runtime

In [ ]:
%pip install python-mecab-ko

KSS 3.7.3 matches python-mecab-ko

In [ ]:
%pip install pandas

In [ ]:
%pip install ray

In [2]:
import re
import os
import kss
import ray
import json
from mecab import MeCab
import pandas as pd
from glob import glob
from itertools import chain

In [3]:
pwd

'D:\\AIHUB'

### Function

In [ ]:
def sorted_list(path_list):
    path_list = sorted(path_list, reverse=False)
    path_list = sorted(path_list, key=len)
    
    return path_list

In [5]:
def json_file_name_list(path_list):
    
    file_name  = [glob(i, recursive = True) for i in path_list][0]
    file_name = sorted_list(file_name)
    
    return file_name

In [6]:
def train_valid_json_file_name_list(path_list):

  train_file_name, valid_file_name = [glob(i, recursive = True) if 'rain' in i
                                      else glob(i, recursive = True)
                                      for i in path_list]

  train_file_name = sorted_list(train_file_name)
  valid_file_name = sorted_list(valid_file_name)
    
  return train_file_name, valid_file_name

In [7]:
def divide_source_file_list(l, n): 
    
  for i in range(0, len(l), n): 
    yield l[i:i + n] 

In [8]:
def txt_file_name_list(source_file_nested_list, folder_corpus_type_name):

  text_file_name_list = [folder_corpus_type_name + str(i) + ".txt"
                              for i in range(len(source_file_nested_list))]
    
  return text_file_name_list

In [9]:
def post_txt_file_name_list(corpus_list):
   
  post_corpus_list = [corpus_file.replace("pro", "post")
                      for corpus_file in corpus_list]

  post_corpus_list = sorted_list(post_corpus_list)

  return post_corpus_list

In [8]:
def formal_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거


    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(source)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = source.find(text) + len(text)
            if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                source = source.replace(small_bracket, text)    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = source.find(text) + len(text)
                if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                    source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    source = source.replace(small_bracket, text)    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        
    
    try:
        ganada_form = re.compile('[가나다라마바사아자차카타파하]+[.]')
        ganada_text = ganada_form.findall(source)

        if type(ganada_text) == str:

            ganada_index = source.find(ganada_text)

            if ganada_index == 0:
                source = re.sub(source[:2], "", source)

        elif type(ganada_text) == list:

            for text in ganada_text:
                ganada_index = source.find(text)

                if ganada_index == 0:
                    source = re.sub(source[:2], "", source)
                    
        source = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", source)

    except:
        pass
             
         # '가.', '나.', ... 형태의 문자열 제거  
        
    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # KSS(Korean Sentence Segmentation)로 문장 분리 
    # Formal articles (wiki, news, essays): recommend to False
    

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, + 제외
            # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
            # 다섯 어절 이하 문장 제외


            if ']' in sentence and '[' not in sentence:
                sentence  = re.sub(r".*?]", "", sentence)    
            # 중괄호 앞에 있는 '성명/직함]' 형태 제거


            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표, 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거

            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외

                for sentence2 in kss.split_sentences(sentence, use_heuristic=False,
                                        num_workers=32):
                    for sentence3 in kss.split_sentences(sentence2, use_heuristic=False,
                                                         num_workers=32):
                        preprocessing_sentence_list.append(sentence3)

            # 마지막에 KSS(Korean Sentence Segmentation)로 문장 분리 2번 실행

  
    return preprocessing_sentence_list

### AIHUB 산업정보 연계 주요국 특허 영-한 데이터

[Source](https://aihub.or.kr/aihubdata/data/view.do?currMenu=&topMenu=&aihubDataSe=realm&dataSetSn=563)

In [11]:
path_list = ['AIHUB_산업정보 연계 주요국 특허 영-한 데이터/Training/'+ '/**/*.json', 
             'AIHUB_산업정보 연계 주요국 특허 영-한 데이터/Validation/'+ '/**/*.json']
train_file_name, valid_file_name = train_valid_json_file_name_list(path_list)

In [12]:
the_number_of_file = len(train_file_name) + len(valid_file_name)
print("The number of file:", the_number_of_file)
n = the_number_of_file // 100
print("The number of list element:", n)
train_file_nested_list = list(divide_source_file_list(train_file_name, n))
valid_file_nested_list = list(divide_source_file_list(valid_file_name, n))

The number of file: 1002
The number of list element: 10


In [16]:
train_text_file_name_list = txt_file_name_list(train_file_nested_list,
                                                           "exploration/british_korean_data_on_patents_in_major_countries_linked_to_industrial_information_pro/AIHUB_british_korean_data_on_patents_in_major_countries_linked_to_industrial_information_train_")
valid_text_file_name_list = txt_file_name_list(valid_file_nested_list,
                                                          "exploration/british_korean_data_on_patents_in_major_countries_linked_to_industrial_information_pro/AIHUB_british_korean_data_on_patents_in_major_countries_linked_to_industrial_information_train_")
the_numer_of_txt_file = len(train_text_file_name_list) + len(valid_text_file_name_list)
print("The number of txt file:", the_numer_of_txt_file)

The number of txt file: 101


In [33]:
train_text_file_name_list[-1]

'exploration/british_korean_data_on_patents_in_major_countries_linked_to_industrial_information_pro/AIHUB_british_korean_data_on_patents_in_major_countries_linked_to_industrial_information_train_49.txt'

In [17]:
def list_length_checker(source_file_nested_list, batch_size):
    
    the_number_of_total_txt_file = 0
    the_number_of_txt_file_list = []
    temp_nested_index = []
    
    for source_file_list in source_file_nested_list:   
        temp_index = []
        for i in range(len(source_file_list)):
            
            if type(source_file_list) == str:
                source_file = source_file_list

            elif type(source_file_list) == list:
                source_file = source_file_list[i]
            

            with open(source_file, 'r', encoding='utf-8') as one_json_file:
                one_json_sample = json.load(one_json_file)
                
            try: 
                source_list = list(pd.DataFrame(one_json_sample['labeled_data'])['astrt_cont_kor'])
            
                the_number_of_txt_file = ((len(source_list) // batch_size) + 1)
                
                if len(source_list) >= 1000:
                    print("File:", source_file)    
                    print("Index:", i, "  ", "Length of Source List:", len(source_list), \
                        "  ", "The number of txt file:", the_number_of_txt_file, "\n")
                    the_number_of_txt_file_list.append(the_number_of_txt_file)
                    the_number_of_total_txt_file  += the_number_of_txt_file
                else:
                    the_number_of_total_txt_file  += 1
                    the_number_of_txt_file_list.append(1)
                    if i > 0 and the_number_of_file % i == 0:
                        temp_index.append(i)
                        try:
                            if i == temp_nested_index[0][0] and len(temp_nested_index) <= 1:
                                print("[For Example]")
                                print("This is not subject of batch. It's small source list.")                            
                                print("File:", source_file)
                                print("Length of Source List:", len(source_list), 
                                      "  ", "The number of txt file:", 1, "\n") 
                        except:
                          pass
            except:
                pass  
            
    print("Batch Size:", batch_size)
    print("The number of txt file:", the_number_of_total_txt_file)
    
    return the_number_of_total_txt_file, the_number_of_txt_file_list

In [18]:
def make_corpus_txt_with_batch_list(source_file_nested_list,
                                    text_file_name_list,
                                    batch_size, the_number_of_txt_file_list):
  
  print("[Size]")
  print("The number of preprocessing corpus: " + str(sum(the_number_of_txt_file_list)))
  print("\n[Order]")
  num = 0
  for i in range(len(source_file_nested_list)):
    source_file_list = source_file_nested_list[i]

    for j in range(len(source_file_list)):

      if type(source_file_list) == str:
        source_file = source_file_list

      elif type(source_file_list) != str:
        source_file = source_file_list[j]

      with open(source_file, 'r', encoding='utf-8') as one_json_file:
        one_json_sample = json.load(one_json_file)

      try:     
        source_list = list(pd.DataFrame(one_json_sample['labeled_data'])['astrt_cont_kor'])
        
        n = batch_size
        source_batch_list = list(divide_source_file_list(source_list, n))
        
        for source_list in source_batch_list:
          
          num += 1
          print(str(num), end=" ")  
          
          with open(os.path.join('AIHUB_corpus/' + text_file_name_list[i][:-4] + "_" + str(num) + ".txt"), "a", encoding='utf-8') as fp:        
              fp.write("\n".join(source_list))   

      except:
        pass     

In [20]:
batch_size = 1000
the_number_of_train_txt_file, the_number_of_train_txt_file_list = list_length_checker(train_file_nested_list, batch_size)

File: AIHUB_산업정보 연계 주요국 특허 영-한 데이터/Training\라벨링데이터_0831_add\TL_1\강선 건조업.json
Index: 4    Length of Source List: 1244    The number of txt file: 2 

File: AIHUB_산업정보 연계 주요국 특허 영-한 데이터/Training\라벨링데이터_0831_add\TL_1\공기 조화장치 제조업.json
Index: 5    Length of Source List: 1183    The number of txt file: 2 

File: AIHUB_산업정보 연계 주요국 특허 영-한 데이터/Training\라벨링데이터_0831_add\TL_1\광학 렌즈 및 광학 요소 제조업.json
Index: 2    Length of Source List: 5024    The number of txt file: 6 

File: AIHUB_산업정보 연계 주요국 특허 영-한 데이터/Training\라벨링데이터_0831_add\TL_1\그 외 기타 분류 안된 화학제품 제조업.json
Index: 2    Length of Source List: 2201    The number of txt file: 3 

File: AIHUB_산업정보 연계 주요국 특허 영-한 데이터/Training\라벨링데이터_0831_add\TL_1\그 외 기타 식료품 제조업.json
Index: 3    Length of Source List: 1348    The number of txt file: 2 

File: AIHUB_산업정보 연계 주요국 특허 영-한 데이터/Training\라벨링데이터_0831_add\TL_1\그 외 기타 의료용 기기 제조업.json
Index: 4    Length of Source List: 5722    The number of txt file: 6 

File: AIHUB_산업정보 연계 주요국 특허 영-한 데이터/Training\라벨링데이터_0831_add\TL

File: AIHUB_산업정보 연계 주요국 특허 영-한 데이터/Training\라벨링데이터_0831_add\TL_5\증류기, 열 교환기 및 가스 발생기 제조업.json
Index: 0    Length of Source List: 1145    The number of txt file: 2 

File: AIHUB_산업정보 연계 주요국 특허 영-한 데이터/Training\라벨링데이터_0831_add\TL_5\축전지 제조업.json
Index: 5    Length of Source List: 2421    The number of txt file: 3 

File: AIHUB_산업정보 연계 주요국 특허 영-한 데이터/Training\라벨링데이터_0831_add\TL_5\컴퓨터 시스템 통합 자문 및 구축 서비스업.json
Index: 5    Length of Source List: 1193    The number of txt file: 2 

File: AIHUB_산업정보 연계 주요국 특허 영-한 데이터/Training\라벨링데이터_0831_add\TL_5\컴퓨터 제조업.json
Index: 6    Length of Source List: 1070    The number of txt file: 2 

File: AIHUB_산업정보 연계 주요국 특허 영-한 데이터/Training\라벨링데이터_0831_add\TL_5\탭, 밸브 및 유사 장치 제조업.json
Index: 6    Length of Source List: 1290    The number of txt file: 2 

File: AIHUB_산업정보 연계 주요국 특허 영-한 데이터/Training\라벨링데이터_0831_add\TL_5\특수직물 및 기타 직물 직조업.json
Index: 1    Length of Source List: 1118    The number of txt file: 2 

File: AIHUB_산업정보 연계 주요국 특허 영-한 데이터/Training\라벨링데이터_0831

In [21]:
batch_size = 1000
make_corpus_txt_with_batch_list(train_file_nested_list, train_text_file_name_list,
                batch_size, the_number_of_train_txt_file_list)

[Size]
The numnber of preprocessing corpus: 640

[Order]
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263

In [22]:
batch_size = 1000
the_number_of_valid_txt_file, the_number_of_valid_txt_file_list = list_length_checker(valid_file_nested_list, batch_size)

File: AIHUB_산업정보 연계 주요국 특허 영-한 데이터/Validation\라벨링데이터_0831_add\VL_4\의약용 화합물 및 항생물질 제조업.json
Index: 0    Length of Source List: 1919    The number of txt file: 2 

File: AIHUB_산업정보 연계 주요국 특허 영-한 데이터/Validation\라벨링데이터_0831_add\VL_5\합성수지 및 기타 플라스틱 물질 제조업.json
Index: 9    Length of Source List: 1161    The number of txt file: 2 

Batch Size: 1000
The number of txt file: 502


In [23]:
batch_size = 1000
make_corpus_txt_with_batch_list(valid_file_nested_list, valid_text_file_name_list,
                batch_size, the_number_of_valid_txt_file_list)

[Size]
The numnber of preprocessing corpus: 502

[Order]
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263

In [24]:
pro_total_corpus_list = glob("AIHUB_corpus/exploration/british_korean_data_on_patents_in_major_countries_linked_to_industrial_information_pro/AIHUB_british_korean_data_on_patents_in_major_countries_linked_to_industrial_information_" +"*.txt")

In [25]:
len(pro_total_corpus_list)

1090

In [26]:
post_total_corpus_list = post_txt_file_name_list(pro_total_corpus_list)

In [27]:
line_list = []
line_num = 0
with open(pro_total_corpus_list[0], 'r', encoding='utf-8') as f:
    lines = f.read().splitlines() 
    for line in lines:
        line_num += 1
        if line_num <= 1:
           line_list.append(line)
for line in line_list:
    print(line, end="\n\n")

본 발명은 매우 우수한 기계적 및 열적 성질을 갖는 탄소, 텅스텐 및 크롬을 포함하는 3D-인쇄된 코발트계 합금 제품 뿐만 아니라 3D-인쇄된 제품의 제조 방법 및 분말 합금에 관한 것이다. 상기 합금은 고 탄소 함량을 가져서 탄화물 함량은 높지만 작고 균일하게 분포된 탄화물을 초래한다. 본 발명의 합금과 같은 고 탄화물 함량 합금의 3D 인쇄를 용이하게 하는 방법 또한 개시되어 있다.



In [28]:
line_list = []
line_num = 0
with open(pro_total_corpus_list[0], 'r', encoding='utf-8') as f:
    lines = f.read().splitlines()
    for line in lines:
        line_num += 1
        if line_num <= 1:  
            sentences = formal_preprocessing_text(line)
            for sentence in sentences:
                line_list.append(sentence) 
            
for line in line_list:
    print(line, end="\n\n")

본 발명은 매우 우수한 기계적 및 열적 성질을 갖는 탄소, 텅스텐 및 크롬을 포함하는 3D-인쇄된 코발트계 합금 제품 뿐만 아니라 3D-인쇄된 제품의 제조 방법 및 분말 합금에 관한 것이다.

상기 합금은 고 탄소 함량을 가져서 탄화물 함량은 높지만 작고 균일하게 분포된 탄화물을 초래한다.

본 발명의 합금과 같은 고 탄화물 함량 합금의 3D 인쇄를 용이하게 하는 방법 또한 개시되어 있다.



In [35]:
ray.init()

@ray.remote

def formal_preprocessing_text(source):
    preprocessing_sentence_list = []
    
    source = source.strip()
    # strip으로 앞뒤 공백 제거

    source = re.sub(r"\[.*?\]|\{.*?\}", "", source)
    # 기타 괄호 제거할 시 괄호 내부에 모든 텍스트 제거


    try:
        bracket_form = re.compile('\(([^)]+)')
        text_in_small_bracket = bracket_form.findall(source)
    
    
        if type(text_in_small_bracket) == str:

            text = text_in_small_bracket

            text_size = len(text)
            last_index = source.find(text) + len(text)
            if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

            if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                small_bracket = "(" + text + ")"
                source = source.replace(small_bracket, text)    

        elif type(text_in_small_bracket) == list:

            for text in text_in_small_bracket:

                text_size = len(text)
                last_index = source.find(text) + len(text)
                if len(source) >= last_index+1 and source[last_index-text_size-1] == '(' and source[last_index+1] == '.':
                    source = source.replace(source[last_index-text_size-1 : last_index+1] + ".", ".")

                if len(text.split()) > 5 and bool(re.match(r'[.]|[!]|[?]', text[-1])) == True:
                    small_bracket = "(" + text + ")"
                    source = source.replace(small_bracket, text)    

    except:
        pass

        # 마침표(.) 앞에 소괄호')'가 있을시 소괄호 제거와 함께 소괄호 내부 텍스트 제거
        # 소괄호 내부 텍스트가 5어절 이상이고 끝이 온점(.). 느낌표(!). 물음표(?)일 떼 소괄호 제거
        
    
    try:
        ganada_form = re.compile('[가나다라마바사아자차카타파하]+[.]')
        ganada_text = ganada_form.findall(source)

        if type(ganada_text) == str:

            ganada_index = source.find(ganada_text)

            if ganada_index == 0:
                source = re.sub(source[:2], "", source)

        elif type(ganada_text) == list:

            for text in ganada_text:
                ganada_index = source.find(text)

                if ganada_index == 0:
                    source = re.sub(source[:2], "", source)
                    
        source = re.sub(r' [가나다라마바사아자차카타파하]+[.]', "", source)

    except:
        pass
             
         # '가.', '나.', ... 형태의 문자열 제거  
        
    for sentence in kss.split_sentences(source, use_heuristic=False,
                                        num_workers=32):
    # KSS(Korean Sentence Segmentation)로 문장 분리 
    # Formal articles (wiki, news, essays): recommend to False
    

        if re.search("^[A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎]", sentence[0]) is not None and \
            bool(re.match(r'[.]|[!]|[?]', sentence[-1])) == True and \
            len(sentence.split()) > 5:
            # 문장의 시작이 특수문자인 문장(영어 대소문자, 한글, 한자, 숫자, -, + 제외
            # 문장의 끝이 온점(.). 느낌표(!). 물음표(?)가 아닌 문장 제외
            # 다섯 어절 이하 문장 제외


            if ']' in sentence and '[' not in sentence:
                sentence  = re.sub(r".*?]", "", sentence)    
            # 중괄호 앞에 있는 '성명/직함]' 형태 제거


            sentence = re.sub(r"[^A-Za-z0-9ㄱ-ㅎ가-힣一-鿕㐀-䶵豈-龎()+-.,]", " ", sentence)
            # 특수문자 제거(영어 대소문자, 한글, 한자, 숫자, -, +, 소괄호, 마침표, 쉼표, 제외)

            sentence = sentence.strip()
            # strip으로 앞뒤 공백 제거

            total_length = len(sentence.replace(" " , ""))
            hangeul_length = len(re.sub(r"[^ㄱ-ㅣ가-힣\s]", "", sentence.replace(" " , "")))
            hangeul_ratio = hangeul_length / total_length
            if hangeul_ratio >= 0.5:
            # 한글이 아닌 문자열이 50% 이상이 넘은 문장 제외

                for sentence2 in kss.split_sentences(sentence, use_heuristic=False,
                                        num_workers=32):
                    for sentence3 in kss.split_sentences(sentence2, use_heuristic=False,
                                                         num_workers=32):
                        preprocessing_sentence_list.append(sentence3)

            # 마지막에 KSS(Korean Sentence Segmentation)로 문장 분리 2번 실행

  
    return preprocessing_sentence_list

2023-05-26 14:49:02,865	INFO worker.py:1625 -- Started a local Ray instance.


In [30]:
print("[Size]")
print("The number of preprocessing corpus: " + str(len(pro_total_corpus_list)))
print("\n[Order]")
num = 0
process_num = 10    

for pro, post in zip(pro_total_corpus_list, post_total_corpus_list):
    
    sentence_list = []
    
    with open(pro, 'r', encoding='utf-8') as f:
        lines = f.read().splitlines() 
        nested_lines_num = len(lines) // process_num
        for i in range(nested_lines_num - 1):
            start_line = process_num * i
            end_line = process_num * (i+1)
            futures = [formal_preprocessing_text.remote(lines[start_line:end_line][j]) for j in range(process_num)]
            results = ray.get(futures)
            
            if i == nested_lines_num - 2:
                futures = [formal_preprocessing_text.remote(lines[end_line:][j]) for j in range(len(lines) - end_line)]
                results = ray.get(futures)
                
            sentences = list(chain.from_iterable(results))
            sentence_list.append(sentences)
    
    sentence_list = list(chain.from_iterable(sentence_list))
    
    num += 1
    print(str(num), end=" ")  
        
    with open(post, 'a', encoding='utf-8') as fp:
        fp.write("\n".join(sentence_list))

[Size]
The numnber of preprocessing corpus: 1090

[Order]
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 

IndexError: list index out of range

In [36]:
ray.shutdown()

In [ ]:
corpus_list = glob("AIHUB_corpus/exploration/british_korean_data_on_patents_in_major_countries_linked_to_industrial_information_post/AIHUB_british_korean_data_on_patents_in_major_countries_linked_to_industrial_information_" +"*.txt")

In [ ]:
with open('AIHUB_corpus/duplicate/AIHUB_british_korean_data_on_patents_in_major_countries_linked_to_industrial_information.txt', 'w') as f:
    for corpus in corpus_list:
        with open(corpus) as text:
            for line in text:
                f.write(line)

In [ ]:
with open('AIHUB_corpus/AIHUB_british_korean_data_on_patents_in_major_countries_linked_to_industrial_information_post/AIHUB_british_korean_data_on_patents_in_major_countries_linked_to_industrial_information.txt', 'w', encoding='utf-8') as f1:
    with open('AIHUB_corpus/duplicate/AIHUB_british_korean_data_on_patents_in_major_countries_linked_to_industrial_information.txt', encoding='utf-8') as f2:
        lines = f2.read().splitlines()
        single_sentence_dict = dict.fromkeys(lines)
        single_sentence_list = list(single_sentence_dict)
        f1.write("\n".join(single_sentence_list))